In [9]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import taxcalc as tc
from taxbrain import (
    TaxBrain, differences_plot, distribution_plot, volcano_plot
)
from bokeh.io import output_notebook, show

ModuleNotFoundError: No module named 'taxbrain'

In [2]:
output_notebook()
pd.options.display.float_format = '{:,.3f}'.format

Loading BokehJS ...

In [3]:
# Set paths to work across Mac/Windows/Linux platforms
# Update cur_path to the path of the repsitory on your local machine
cur_path = "/Users/richardevans/Docs/Economics/OSE/Federal/CharitableContrib"
data_dir = os.path.join(cur_path, 'data')
images_dir = os.path.join(cur_path, 'images')
# Directories to save data
json_dir = os.path.join(cur_path, "json")
base_dir = os.path.join(cur_path, "OUTPUT_BASELINE")
reform_dir_chddct = os.path.join(cur_path, "OUTPUT_REFORM_CHDDCT")
tmd_dir = (
    "/Users/richardevans/Docs/Economics/OSE/microsim/" +
    "tax-microdata-benchmarking/tmd/storage/output"
)

## 2. Run Tax-Calculator analysis

### 2.1. Run TaxBrain using TMD with TMD_dict

In [4]:
start_year = 2025
end_year = 2034
base2_dict = tc.Policy.read_json_reform(os.path.join(json_dir, "ext.json"))
ref_dict = tc.Policy.read_json_reform(os.path.join(json_dir, "chddct.json"))

In [5]:
start_year = 2025
end_year = 2034
base_dict = tc.Policy.read_json_reform(os.path.join(json_dir, "ext.json"))
ref_dict = tc.Policy.read_json_reform(os.path.join(json_dir, "chddct.json"))
TMD_dict = {
        "data": os.path.join(tmd_dir, "tmd_jason.csv.gz"),
        "start_year": start_year,
        "growfactors": os.path.join(tmd_dir, "tmd_growfactors_jason.csv"),
        "weights": os.path.join(tmd_dir, "tmd_weights_jason.csv.gz"),
}
tb_static_1 = TaxBrain(
    start_year=start_year, end_year=end_year, microdata=TMD_dict, base_policy={}, reform=ref_dict
)
tb_static_1.run()
print("Combined Tax Liability Over the Budget Window")
print(
    (tb_static_1.weighted_totals("combined", include_total=True) / 1e9)
    .to_markdown()
)  # division to put in billions

Combined Tax Liability Over the Budget Window
|            |       2025 |        2026 |        2027 |        2028 |        2029 |        2030 |        2031 |        2032 |        2033 |        2034 |       Total |
|:-----------|-----------:|------------:|------------:|------------:|------------:|------------:|------------:|------------:|------------:|------------:|------------:|
| Base       | 3575.24    | 3972.7      | 4130.66     | 4295.57     | 4460.89     | 4649.55     | 4846.01     | 5047.26     | 5246.97     | 5458.56     | 45683.4     |
| Reform     | 3571.88    | 3972.05     | 4130.04     | 4294.98     | 4460.34     | 4649.02     | 4845.51     | 5046.78     | 5246.52     | 5458.13     | 45675.2     |
| Difference |   -3.35566 |   -0.652049 |   -0.621828 |   -0.587704 |   -0.555889 |   -0.527882 |   -0.501602 |   -0.477335 |   -0.455969 |   -0.436817 |    -8.17274 |


### 2.2. Run Tax-Calculator using TMD inputting directly to records

In [6]:
# Set up the baseline calculator
gfactors=tc.growfactors.GrowFactors(
    growfactors_filename=os.path.join(tmd_dir, "tmd_growfactors_jason.csv")
)
base_pol_1 = tc.policy.Policy(gfactors=gfactors, last_budget_year=end_year)
base_pol_2 = tc.policy.Policy(gfactors=gfactors, last_budget_year=end_year)
base_pol_2.implement_reform(base_dict)
# rec = tc.records.Records.cps_constructor()
rec = tc.records.Records(
    data=os.path.join(tmd_dir, "tmd_jason.csv.gz"),
    start_year=tc.records.Records.TMDCSV_YEAR,
    gfactors=gfactors,
    weights=os.path.join(tmd_dir, "tmd_weights_jason.csv.gz"),
    adjust_ratios=None,
)
tc_base_1 = tc.calculator.Calculator(policy=base_pol_1, records=rec)
print("taxcalc TMD start year is:", tc.records.Records.TMDCSV_YEAR)

taxcalc TMD start year is: 2021


In [7]:
ref_pol_1 = tc.policy.Policy(gfactors=gfactors, last_budget_year=end_year)
# ref_pol.implement_reform(base1_dict)
ref_pol_1.implement_reform(ref_dict)
tc_ref_1 = tc.calculator.Calculator(policy=ref_pol_1, records=rec)

In [8]:
# Compute annual revenue for each year in a 10-year budget window
# Do this for baseline and reforms
rev_base_1 = {'IIT': {}, 'Payroll': {}, 'Sum': {}}
rev_ref_1 = {'IIT': {}, 'Payroll': {}, 'Sum': {}}
for t in range(start_year, end_year+1):
    tc_base_1.advance_to_year(t)
    tc_base_1.calc_all()
    rev_base_1['IIT'][t] = tc_base_1.weighted_total('iitax')
    rev_base_1['Payroll'][t] = tc_base_1.weighted_total('payrolltax')
    rev_base_1['Sum'][t] = rev_base_1['IIT'][t] + rev_base_1['Payroll'][t]
    tc_ref_1.advance_to_year(t)
    tc_ref_1.calc_all()
    rev_ref_1['IIT'][t] = tc_ref_1.weighted_total('iitax')
    rev_ref_1['Payroll'][t] = tc_ref_1.weighted_total('payrolltax')
    rev_ref_1['Sum'][t] = rev_ref_1['IIT'][t] + rev_ref_1['Payroll'][t]

In [9]:
# make table of revenue estimates
rev_base_1_df = pd.DataFrame.from_dict(rev_base_1).T
rev_base_1_df['2025-2034'] = rev_base_1_df.sum(axis=1)
rev_ref_1_df = pd.DataFrame.from_dict(rev_ref_1).T
# rev_ref_df['2025-2034'] = rev_ref_df.sum(axis=1)
diff_1_df = (rev_ref_1_df - rev_base_1_df) * 1e-9
pd.options.display.float_format = '${:.3f}'.format
diff_1_df['2025-2034'] = diff_1_df.sum(axis=1)
print('Charitable Act revenue estimates')
print(rev_base_1_df * 1e-9)
print(rev_ref_1_df * 1e-9)
print(diff_1_df.to_markdown())

Charitable Act revenue estimates
             2025      2026      2027      2028      2029      2030      2031  \
IIT     $2580.483 $3005.681 $3134.365 $3269.359 $3399.354 $3551.310 $3709.749   
Payroll $1706.931 $1785.590 $1859.037 $1930.053 $2003.975 $2080.308 $2158.496   
Sum     $4287.415 $4791.270 $4993.401 $5199.412 $5403.328 $5631.618 $5868.244   

             2032      2033      2034  2025-2034  
IIT     $3873.742 $4041.445 $4213.046 $34778.534  
Payroll $2237.907 $2318.438 $2400.162 $20480.896  
Sum     $6111.649 $6359.883 $6613.208 $55259.429  
             2025      2026      2027      2028      2029      2030      2031  \
IIT     $2577.064 $3005.293 $3133.995 $3269.007 $3399.013 $3550.981 $3709.429   
Payroll $1706.931 $1785.590 $1859.037 $1930.053 $2003.975 $2080.308 $2158.496   
Sum     $4283.995 $4790.882 $4993.031 $5199.060 $5402.988 $5631.289 $5867.924   

             2032      2033      2034  
IIT     $3873.427 $4041.135 $4212.740  
Payroll $2237.907 $2318.438 $2400

### 2.1. Setup baseline calculator

In [ ]:
# Set up the baseline calculator
base_pol = tc.policy.Policy()
# base_pol_dict = base_pol.read_json_reform(os.path.join(json_dir, "ext.json"))
# base_pol.implement_reform(base_pol_dict)
gfactors=tc.growfactors.GrowFactors(
    growfactors_filename=os.path.join(tmd_dir, "tmd_growfactors_jason.csv")
)
# rec = tc.records.Records.cps_constructor()
rec = tc.records.Records(
    data=os.path.join(tmd_dir, "tmd_jason.csv.gz"),
    start_year=tc.records.Records.TMDCSV_YEAR,
    gfactors=gfactors,
    weights=os.path.join(tmd_dir, "tmd_weights_jason.csv.gz"),
    adjust_ratios=None,
)
tc_base = tc.calculator.Calculator(policy=base_pol, records=rec)
print("taxcalc TMD start year is:", tc.records.Records.TMDCSV_YEAR)


### 2.2. Setup reform calculator

In [ ]:
ref_pol = tc.policy.Policy()
ref_pol.implement_reform(base_pol_dict)
ref_pol_dict = base_pol.read_json_reform(os.path.join(json_dir, "chddct.json"))
ref_pol.implement_reform(ref_pol_dict)
tc_ref = tc.calculator.Calculator(policy=ref_pol, records=rec)

### 2.3. Create the revenue estimates for 10-year budget window

In [ ]:
# Compute annual revenue for each year in a 10-year budget window
# Do this for baseline and reforms
start_year = 2025
end_year = 2034

rev_base = {'IIT': {}, 'Payroll': {}, 'Sum': {}}
rev_ref = {'IIT': {}, 'Payroll': {}, 'Sum': {}}
for t in range(start_year, end_year+1):
    tc_base.advance_to_year(t)
    tc_base.calc_all()
    rev_base['IIT'][t] = tc_base.weighted_total('iitax')
    rev_base['Payroll'][t] = tc_base.weighted_total('payrolltax')
    rev_base['Sum'][t] = rev_base['IIT'][t] + rev_base['Payroll'][t]
    tc_ref.advance_to_year(t)
    tc_ref.calc_all()
    rev_ref['IIT'][t] = tc_ref.weighted_total('iitax')
    rev_ref['Payroll'][t] = tc_ref.weighted_total('payrolltax')
    rev_ref['Sum'][t] = rev_ref['IIT'][t] + rev_ref['Payroll'][t]

### 2.4. Create a table of output

In [ ]:
# make table of revenue estimates
rev_base_df = pd.DataFrame.from_dict(rev_base).T
rev_base_df['2025-2034'] = rev_base_df.sum(axis=1)
rev_ref_df = pd.DataFrame.from_dict(rev_ref).T
# rev_ref_df['2025-2034'] = rev_ref_df.sum(axis=1)
diff_df = (rev_ref_df - rev_base_df) * 1e-9
pd.options.display.float_format = '${:.3f}'.format
diff_df['2025-2034'] = diff_df.sum(axis=1)
print('Charitable Act revenue estimates')
print(rev_base_df * 1e-9)
print(rev_ref_df * 1e-9)
print(diff_df.to_markdown())

## 3. Charitable Donations response

In [4]:
# GivingUSA data

cd_2019 = 434.03
cd_2020 = 478.94
cd_2021 = 552.51

diff_2020 = cd_2020 - cd_2019
diff_2021 = cd_2021 - cd_2020

print("2020 change in charitable donations:", diff_2020)
print("15% of 2020 change in charitable donations:", diff_2020 * 0.15)
print("")
print("2021 change in charitable donations:", diff_2021)
print("15% of 2021 change in charitable donations:", diff_2021 * 0.15)

2020 change in charitable donations: 44.910000000000025
15% of 2020 change in charitable donations: 6.736500000000004

2021 change in charitable donations: 73.57
15% of 2021 change in charitable donations: 11.035499999999999


In [13]:
import numpy as np
print(
    f"In 2020, every $100 of extra exeption generated $",
    np.round(((diff_2020 * 0.15)/300) * 1e3, decimals=2),
    "million of extra charitable donations."
)
print(
    f"In 2021, every $100 of extra exeption generated $",
    np.round(((diff_2021 * 0.15)/600) * 1e3, decimals=2),
    "million of extra charitable donations."
)

In 2020, every $100 of extra exeption generated $ 22.46 million of extra charitable donations.
In 2021, every $100 of extra exeption generated $ 18.39 million of extra charitable donations.


In [ ]:
100*100*4

183900.0

In [17]:
0.1433*10000+2.4375

1435.4375